In [1]:
import math
from math import pi, sqrt, tanh
from cmath import exp
import numpy as np
import kwant
from matplotlib import pyplot as plt

D:\Programme\Anaconda\lib\site-packages\kwant\solvers\default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


In [2]:
%matplotlib inline

In [3]:
def onsite_wrapper(delta_sign):
    '''Gives the onsite term depending on the sign of the gate'''
    def onsite(site, params):
        print(delta_sign, params['delta'])
        return params['onsite'] + delta_sign*params['delta']
    return onsite

def generic_hopping(name):
    '''Wrapper for a constant generic hopping defined in the parameters'''
    def hopping(site_i, site_j, params):
        return params[name]
    return hopping

def zero(x):
    '''Utility'''
    return 0

In [4]:
def shape(start = None, end = (0,0,0)):
    '''Defines a quadratic shape. If no start is given, generates the shape by point reflection around 0'''
    Lx, Ly, Lz = end
    
    if start is None:
        Sx, Sy, Sz = -np.array(end)
    else:
        Sx, Sy, Sz = start
        
    def rectangle(pos):
        x,y,z = pos
        return Sx <= x <= Lx and Sy <= y <= Ly and Sz <= z <= Lz
    return rectangle 

In [5]:
def generate_system(lattice, hoppings, hopping_conditions, onsite_wrapper, shape1, axis1 = (0,0,0), shape2 = None, axis2 = (0,0,0), interface_length = 0, symmetry = None):
    '''Can generate either scattering region or lead depending on the symmetry parameter'''
    if symmetry is None:
        sys = kwant.Builder()
    else:
        sys = kwant.Builder(kwant.TranslationalSymmetry(symmetry))        
    if shape2 is None:
        sys[lattice.shape(shape1, axis1)] = onsite_wrapper(0)   
    else:
        sys[lattice.shape(shape1, axis1)] = onsite_wrapper(1)
        sys[lattice.shape(shape2, axis2)] = onsite_wrapper(-1)
        
    for hopping, hopping_condition in zip(hoppings, hopping_conditions):
        sys[[kwant.builder.HoppingKind(*h) for h in hopping_condition]] = hopping
    return sys

In [6]:
def colors(a1, a2, b1, b2):
    '''Defines site colors for a bilayer graphene system with 4 sites A1, A2, B1, B2'''
    def family_colors(site):
        if site.family == a1:
            return 'b'
        elif site.family == a2:
            return 'k'
        elif site.family == b1:
            return 'y'
        elif site.family == b2:
            return 'g'
        else:
            raise ValueError('atom has to be on one of the given sites')
    return family_colors

In [7]:
def generate_lattice(stacking):
    '''Just generates a graphene bilayer lattice with some default geometry and user defined stacking'''
    a_L=1
    c_L=1.36178862*a_L
    sin_30, cos_30 = (1 / 2, sqrt(3) / 2)
    
    if str.upper(stacking) == "AB":
        vec_len = -a_L * 999/1000
    elif str.upper(stacking) == "BA":
        vec_len = a_L * 999/1000
    else:
        raise ValueError("Stacking can only be AB or BA")
        
    graphene = kwant.lattice.general([(a_L, 0, 0), ( a_L*0.5, a_L*sqrt(3)*0.5, 0), (vec_len, 0, 2*c_L)], 
                                 [(0, 0, -c_L/2), (0, a_L / sqrt(3), -c_L/2), (vec_len, 0, c_L/2), (a_L*0.5 + vec_len, a_L * 0.5 /sqrt(3), c_L/2) ])
    return graphene

In [8]:
def build_hoppings(graphene, stacking, Lx, Ly, Lz):
    '''Uses the given hppings and lattice to generate the hoppings in the system builder'''
    
    a1, b1, a2, b2 = graphene.sublattices
    hoppinga = generic_hopping('hoppingIntra')
    hoppingb = generic_hopping('hoppingIntra')
    hoppingabba = generic_hopping('hoppingInter')
    
    hopping_a = (((0, 0, 0), a1, b1), ((0, 1, 0), a1, b1), ((-1, 1, 0),a1, b1))
    hopping_b = (((0, 0, 0), a2, b2), ((0, 1, 0), a2, b2), ((1, 0, 0), a2,b2))
    
    #Depening on the shift of the upper lattice, the site enumeration must also be shifted
    if str.upper(stacking) == "AB":
        hopping_abba = [((0, 0, 0), a1, b2),((1, 0, 0), a2, b1)]
    elif str.upper(stacking) == "BA":
        hopping_abba = [((1, 0, 0), a1, b2),((0, 0, 0), a2, b1)]
    else:
        raise ValueError("Stacking can only be AB or BA")
   
    hoppings = [hoppinga, hoppingb, hoppingabba]
    hopping_conditions = [hopping_a, hopping_b, hopping_abba]
    
    return hoppings, hopping_conditions

In [9]:
def plot_conductance(fsys, args, energies = np.linspace(-15, 15, 100)):
    '''Calculates and plots the conductance for a finaluzed system and an energy range'''
    data = []

    for energy in energies:
        smatrix = kwant.smatrix(fsys, energy, args = args)
        data.append(smatrix.transmission(1, 0))
    
    plt.figure()
    plt.xlabel("energy (t)")
    plt.ylabel("conductance (e^2/h)")
    plt.plot(energies, data)

In [10]:
def create_arguments(hopping_parameters = [3.16, 0.4], delta = 1):
    '''Create the argument list in the format required'''
    return ({'onsite':0, 'hoppingIntra':hopping_parameters[0], 'hoppingInter':hopping_parameters[1], 'delta':delta},)

In [20]:
def make_system(stacking, symmetry, dimensions, offset = (0,0,0), plot = False, leads = True):
    '''Build the whole system with leads and scattering region'''
    #BILAYER graphene
    Lz = 2
    Lx, Ly = dimensions
 
    #generate lattice and hoppings
    graphene = generate_lattice(stacking)
    a1, b1, a2, b2 = graphene.sublattices
    hoppings, hopping_conditions = build_hoppings(graphene, stacking, Lx, Ly, Lz)
    
    #Different onsite terms for upper and lower layer in case of gating. Apply hoppings and onsite terms
    ll_shape = shape(end = (Lx, Ly, 0), start = (-Lx, -Ly, -1))
    ul_shape = shape(end = (Lx, Ly, 1), start = (-Lx, -Ly, 0))
    system = generate_system(graphene, hoppings, hopping_conditions, onsite_wrapper, shape1 = ll_shape, shape2 = ul_shape, axis1 = (0, 0, -1), axis2 = (0, 0, 0))
    
    if leads:
        #Zigzag and armchair are just differentiated by the position of the leads
        if str.lower(symmetry) == "zigzag":
            left_lead = generate_system(graphene, hoppings, hopping_conditions, zero, shape1 = shape(end = (np.infty, Ly, Lz)), interface_length = Ly, symmetry = graphene.vec((-1,0,0)))
            right_lead = generate_system(graphene, hoppings, hopping_conditions, zero, shape1 = shape(end = (np.infty, Ly, Lz)), interface_length = Ly, symmetry = graphene.vec((1,0,0)))
        elif str.lower(symmetry) == "armchair":
            left_lead = generate_system(graphene, hoppings, hopping_conditions, zero, shape1 = shape(end = (Lx, np.infty, Lz)), interface_length = Lx, symmetry = graphene.vec((1,-2,0)))
            right_lead = generate_system(graphene, hoppings, hopping_conditions, zero, shape1 = shape(end = (Lx, np.infty, Lz)), interface_length = Lx, symmetry = graphene.vec((-1,2,0)))
        else:
            raise ValueError("Symmetry must be either zigzag or armchair")
    
        #Attach the defined leads to the system
        system.attach_lead(left_lead)
        system.attach_lead(right_lead)    
    
    if plot:
        kwant.plot(system, site_size=0.18, site_lw=0.01, hop_lw=0.05, site_color = colors(a1, a2, b1, b2), pos_transform = (lambda pos: [pos[0], pos[1], 0]))
        kwant.plot(system, site_size=0.18, site_lw=0.01, hop_lw=0.05, site_color = colors(a1, a2, b1, b2))
        kwant.plot(system, site_size=0.18, site_lw=0.01, hop_lw=0.05, site_color = colors(a1, a2, b1, b2))
    return system.finalized()

Lx = 20
Ly = 20

In [12]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [21]:
sys1 = make_system("AB", "armchair", (Lx, Ly))
def f(x):
    arguments = create_arguments(hopping_parameters = [3.0, x],delta = 0)
    kwant.plotter.bands(sys1.leads[0], args = arguments)
    print(x)
interact(f, x = 3.0);

In [17]:
sys1 = make_system("AB", "zigzag", (Lx, Ly))
interact(f, x = 3.0);